In [1]:
import pandas as pd
import bamboolib as bam
import sqlite3
from sqlite3 import Error
import sqlalchemy as sqla
from thefuzz import fuzz
from thefuzz import process

## Thefuzz

In [5]:
process.extractOne("uns",['uns cnrs observatoire cote azur'])[1]

60

In [14]:
fuzz.partial_ratio("universite cote azur", ['departement enseignement recherche medecine', 'faculte medecine', 'universite sophia antipolis'])

95

In [47]:
l = ['artificial intelligence institute', "skema business school cote azur university"]
[s.replace("'","") for s in l]

['artificial intelligence institute',
 'skema business school cote azur university']

In [48]:
max(process.extractOne("universite cote azur",l)[1],
               process.extractOne("university cote azur",l)[1], 
               process.extractOne("universite azur",l)[1],
               process.extractOne("cote azur university",l)[1], 
               fuzz.partial_ratio("univ cote azur",l),
               process.extractOne("universitcrossed sign cote azur",l)[1],
               process.extractOne("universitcrossed cote azur",l)[1],
               process.extractOne("universit x00e9 c x00f4 te x2019 azur",l)[1])

90

In [110]:
max(process.extractOne("universite nice",l)[1],
    process.extractOne("universite nice sophia",l)[1],
    process.extractOne("nice sophia antipolis",l)[1],
    process.extractOne("universite sophia antipolis",l)[1],
    process.extractOne("university nice",l)[1],
    process.extractOne("university nice sophia",l)[1],
    process.extractOne("university sophia antipolis",l)[1],
    process.extractOne("sophia antipolis university",l)[1],
    process.extractOne("antipolis university",l)[1],
    process.extractOne("nice university",l)[1],
    process.extractOne("univ nice",l)[1],
    fuzz.partial_ratio("univ nice",l),
    fuzz.partial_ratio("uns",l))

95

In [3]:
process.extractOne("university nice",['university nice sophia antipolis'])

('university nice sophia antipolis', 90)

## Sqlite3

In [18]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

create_connection(r"../../../publications.db")

2.6.0


In [2]:
conn = sqlite3.connect(r"../../../publications.db")
cur = conn.cursor()
#cur.execute("CREATE TABLE IF NOT EXISTS bso_publis_all_by_affiliation_20201130 ('source_id', 'doi', 'scopus_title', 'year', 'aff_scopus_id','aff_source_text', 'author_id', 'author_name', 'corresponding_author','creator_author', 'all_authors', 'affiliation-name', 'aff_internal_id','aff_parent_id', 'corresponding')")

In [56]:
df2.to_sql("bso_publis_all_by_affiliation_20201130", conn, if_exists='append', index=False)

In [6]:
cur.execute("select id,affiliation_name,document_count from referentiel_structures_20220829 where parent_id != 0")
rows = cur.fetchall()
d = pd.DataFrame(rows)

OperationalError: no such column: document_count

In [5]:
d.columns = [str(column) for column in d.columns]
d

,0
0,10.1038/s41467-019-12568-w
1,10.1007/978-3-030-69480-7_9


In [34]:
df = pd.read_csv("publis_uniques_doi_oa_data_with_bsoclasses_complete.csv", sep=',', encoding="utf-8", usecols=["doi", "year", "genre", "is_oa_normalized","oa_status_normalized"], dtype={"doi": str, "year": str})

## SQLAlchemy

In [2]:
dbEngine=sqla.create_engine('sqlite:////home/docker/app/bso_publis_scopus/09_db/publications.db')

In [8]:
PUBLIS_OBS_DATES = ["20220829","20210914","20201130"]
def get_publis_dataset(publis_obs_date):
    df = pd.read_sql(f'select doi, year, genre, is_oa_normalized, oa_status_normalized, oa_host_type_normalized from bso_publis_uniques_{publis_obs_date}',dbEngine)
    return df
def get_crosstab_percent(df, col1, col2):
    c = (pd.crosstab(df[col1], df[col2], normalize='index')*100).round(1).reset_index()
    return c

In [ ]:
comp_last_date = "20220829"
comp_first_date = "20201130"

In [9]:
df = pd.read_sql(f'select distinct(bso_publis_uniques_{comp_last_date}.doi) as doi, bso_publis_uniques_{comp_last_date}.year, bso_publis_uniques_{comp_last_date}.title, '
                       f'bso_publis_uniques_{comp_last_date}.publisher_by_doiprefix, bso_publis_uniques_{comp_last_date}.is_oa_normalized, bso_publis_uniques_{comp_last_date}.oa_status_normalized '
                       f'FROM bso_publis_uniques_{comp_last_date} JOIN bso_publis_uniques_{comp_first_date} ON bso_publis_uniques_{comp_last_date}.doi = bso_publis_uniques_{comp_first_date}.doi '
                       f'JOIN bso_publis_all_by_affiliation_{comp_last_date} ON bso_publis_uniques_{comp_last_date}.doi = bso_publis_all_by_affiliation_{comp_last_date}.doi '
                       f'WHERE bso_publis_all_by_affiliation_{comp_last_date}.aff_internal_id = {selected_structure} AND bso_publis_uniques_{comp_first_date}.is_oa_normalized = "{comp_first_oa_state}" '
                       f'AND bso_publis_uniques_{comp_last_date}.is_oa_normalized <> bso_publis_uniques_{comp_first_date}.is_oa_normalized' ,dbEngine)

is_oa_normalized,year,Accès fermé,Accès ouvert,year_obs
0,2016,39.1,60.9,2022
1,2017,35.4,64.6,2022
2,2018,34.6,65.4,2022
3,2019,29.7,70.3,2022
4,2020,27.3,72.7,2022
5,2021,27.2,72.8,2022
0,2016,39.3,60.7,2021
1,2017,35.9,64.1,2021
2,2018,35.1,64.9,2021
3,2019,32.4,67.6,2021
